In [ ]:
## analysing 16s faecal microbiome data from wombat scat samples using Qiime2. (V2024.2)

In [ ]:
## first we will import our data using the import manifest method. 

In [ ]:
## we only imported the R1 sequences following DOI: 10.1128/AEM.00062-07 (figure 1)

In [ ]:
qiime tools import \
--type 'SampleData[SequencesWithQuality]' \
--input-path /hpcfs/users/a1805545/16s_reads/import-manifest-11072024.tsv \
--output-path /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW_SHNW_all_16s.qza \
--input-format SingleEndFastqManifestPhred33V2 

In [ ]:
## once imported we will use the deblur method to generate features (OTU's/ASV's)
## set the trim length 

In [ ]:
qiime deblur denoise-16S \
--i-demultiplexed-seqs /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW_all_16s.qza \
--p-trim-length 150 \
--p-sample-stats \
--o-representative-sequences /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-rep-seqs.qza \
--o-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-table.qza \
--o-stats /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-deblur-stats.qza \
--verbose 

In [ ]:
## then we will visualize the results

In [ ]:
qiime feature-table tabulate-seqs \
--i-data /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-rep-seqs.qza\
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-rep-seqs.qzv

In [ ]:
qiime feature-table summarize \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-table.qza \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-table.qzv

In [ ]:
## following this we created a phylogenetic tree using SEPP.

In [ ]:
qiime fragment-insertion sepp \
--i-representative-sequences /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-rep-seqs.qza \
--i-reference-database /hpcfs/users/a1805545/16s_reads/classifier/sepp-refs-silva-128.qza \
--o-tree /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-sepp-tree.qza \
--o-placements /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-sepp-placements.qza \
--verbose 


In [ ]:
## next we mapped our reads to the tree and removed any that didnt map.

In [ ]:
qiime fragment-insertion filter-features \
  --i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table.qza \
  --i-tree /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-sepp-tree.qza \
  --o-filtered-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-filtered-table.qza \
  --o-removed-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-removed-table.qza \
  --verbose

In [ ]:
## all features mapped so we will use the unfiltered table.

In [ ]:
## now we will perform an alpha rarefaction to determine an appropriate sampling depth.

In [ ]:
qiime diversity alpha-rarefaction \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--o-visualization NHNW-SHNW-faecal-table-rarefaction.qzv \
--p-max-depth 56000 

In [ ]:
## Looks like 25K is where most samples level out will use that as our sampling depth.

In [ ]:
## Now we will reove samples following the SHNW 16s analysis, soil and NTCs.

In [ ]:
qiime feature-table filter-samples \
--i-table  /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-where "Species IN ('Soil', 'NTC')" \
--p-exclude-ids \
--o-filtered-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil.qza

In [ ]:
## now we will remove any samples that dont have atleast 25K reads. 

In [ ]:
qiime feature-table filter-samples \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-min-frequency 25000 \
--o-filtered-table NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qza

In [1]:
## convert the table to viewable to make sure everything has been removed correctly. 

In [ ]:
qiime feature-table summarize \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qza \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qzv \

In [ ]:
## next we will assign taxonomy using SKlearn and the SILVA database. 

In [ ]:
qiime feature-classifier classify-sklearn \
--i-reads /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-rep-seqs.qza \
--i-classifier /hpcfs/users/a1805545/16s_reads/classifier/silva-139-99-515-806-classifier.qza \
--o-classification /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNE-22-23-SILVA-taxa.qza

In [ ]:
## we will make a barplot with the taxa assignments.

In [ ]:
qiime taxa barplot \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qza \
--i-taxonomy /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-22-23-SILVA-taxa.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-22-23-SILVA-taxa-Barplot-25000-reads.qzv

In [ ]:
## next we will remove duplicates, joey and contaminated samples, according to the SHNW 16s analysis

In [ ]:
qiime feature-table filter-samples \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-where  "name IN ('B10-SRR12658844', 'B11-SRR12658843', 'B13-SRR12658841', 'B14-SRR12658840', 'B15-SRR12658839', 'B16-SRR12658838', 'B17-SRR12658837', 'M7b-SRR12658823', 'M6-SRR12658826', 'M14-SRR12658813', 'M1b-SRR12658834', 'M1c-SRR12658833', 'M3b-SRR12658830', 'M4b-SRR12658828', 'M9b-SRR12658820')" \
--p-exclude-ids \
--o-filtered-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-atleast-25000-reads.qza

In [3]:
## now we will make new barplots with samples removed.

In [ ]:
qiime taxa barplot \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-atleast-25000-reads.qza \
--i-taxonomy /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-22-23-SILVA-taxa.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-barplot-no-soil-atleast-25000-reads.qzv

In [4]:
# lets collapse the table into populations to get an idea of larger trends. 

In [ ]:
qiime feature-table group \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-atleast-25000-reads.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column cluster \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-cluster.qza

In [5]:
## visualise condensed table to ensure correct metadata format. 

In [ ]:
qiime feature-table summarize \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output//NHNW-SHNW-final-faecal-table-cluster.qza \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output//NHNW-SHNW-final-faecal-table-cluster.qzv \

In [ ]:
## Make barplots of the clusters                                        

In [ ]:
qiime taxa barplot \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-cluster.qza \
--i-taxonomy /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-22-23-SILVA-taxa.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata-cluster.txt \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-cluster-faecal-barplot-no-soil-atleast-25000-reads.qzv

In [ ]:
## now we will run the core diversity metrics on all samples (non collapsed table)

In [ ]:
qiime diversity core-metrics-phylogenetic \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-barplot-no-soil-atleast-25000-reads.qza \
--i-phylogeny /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-sepp-tree.qza \
--p-sampling-depth 25000 \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--output-dir /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-core-div

In [ ]:
## looks like there some separation as well as some outlier samples lets run PERMANOVA to look into the community difference

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-core-div/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column reserve \
--o-visualization NHNW-SHNW-unweighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-core-div/weighted_unifrac_distance_matrix.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column reserve \
--o-visualization NHNW-SHNW-weighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
## lets also run this as a difference between species not just cluster

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-core-div/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column Species \
--o-visualization NHNW-SHNW-species-unweighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-core-div/weighted_unifrac_distance_matrix.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column Species \
--o-visualization NHNW-SHNW-species-weighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
## we should remove the captive wombats from future analysis as we know hey have different communities. 

In [ ]:
qiime feature-table filter-samples \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-atleast-25000-reads.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-where "cluster IN ('Captive Adelaide Hills')" \
--p-exclude-ids \
--o-filtered-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-captive-atleast-25000-reads.qza

In [ ]:
## now lets re-run the core diversity without tthe captive population to understand how wild populations only compare.

In [ ]:
qiime diversity core-metrics-phylogenetic \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-captive-atleast-25000-reads.qza \
--i-phylogeny /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-sepp-tree.qza \
--p-sampling-depth 25000 \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--output-dir /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div

In [ ]:
## now we will run the PERMANOVA on the core diversity without the captive population. Species level

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column Species \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div/NHNW-SHNW-wild-species-unweighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div/weighted_unifrac_distance_matrix.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column Species \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div/NHNW-SHNW-wild-species-weighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
## We will also run a PERMANOVA on the wild reserves

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div/weighted_unifrac_distance_matrix.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column reserve \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div/NHNW-SHNW-wild-reserve-weighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column reserve \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-wild-core-div/NHNW-SHNW-wild-reserve-unweighted_unifrac_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
## The next step will be ANCOM to identiy differentially abundant features in our populations. 
## First we will fliter out the low abundance features in the feature table. 

In [ ]:
qiime feature-table filter-features \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-captive-atleast-25000-reads.qza \
--o-filtered-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500.qza  \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
## lets view our filtered table and compare it to the other final table.

In [ ]:
qiime feature-table summarize \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500.qza  \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500.qzv

In [ ]:
qiime feature-table summarize \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-captive-atleast-25000-reads.qza \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-captive-atleast-25000-reads.qzv

In [ ]:
## To make these tables more interpratable we need to collapse the feature table into the nearest taxonomies we will use lvl 5 which is family.

In [ ]:
qiime taxa collapse \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500.qza \
--p-level 5 \
--i-taxonomy /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-SILVA-taxa.qza \
--o-collapsed-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500-lvl5.qza

In [ ]:
## Lets make a genus table as well

In [ ]:
qiime taxa collapse \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500.qza \
--p-level 6 \
--i-taxonomy /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-SILVA-taxa.qza \
--o-collapsed-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500-lvl6.qza

In [ ]:
## We will now re run with ANCOMBC (differential abundance with bias correction) using the table with collapsed taxonomy

In [ ]:
qiime composition ancombc \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500-lvl5.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-formula Species \
--o-differentials /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOMBC-ms8-mf-500-spceies.qza 

In [ ]:
## lets also run this at the genus level for a bit more detail.

In [ ]:
qiime composition ancombc \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500-lvl6.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-formula Species \
--o-differentials /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOMBC-ms8-mf-500-lvl6-spceies.qza 

In [ ]:
## to compare the reserve we want to use EFNP as the reference. 

In [ ]:
qiime composition ancombc \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500-lvl5.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-formula reserve \
--p-reference-levels reserve::EFNP \
--o-differentials /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOMBC-ms8-mf-500-lvl5-reserve.qza

In [ ]:
qiime composition ancombc \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOM-ms8-mf-500-lvl6.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-formula reserve \
--p-reference-levels reserve::EFNP \
--o-differentials /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOMBC-ms8-mf-500-lvl6-reserve.qza

In [1]:
## visualising the ancombc results use label limit to make the taxa string readable

In [ ]:
qiime composition da-barplot \
--i-data /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOMBC-ms8-mf-500-spceies.qza \
--p-level-delimiter ';' \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-ANCOMBC-ms8-mf-500-species.qzv

In [ ]:
qiime composition da-barplot \
--i-data /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOMBC-ms8-mf-500-lvl5-reserve.qza \
--p-level-delimiter ';' \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-ANCOMBC-ms8-mf500-lvl5-reserve.qzv

In [ ]:
qiime composition da-barplot \
--i-data /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOMBC-ms8-mf-500-lvl6-spceies.qza \
--p-level-delimiter ';' \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-ANCOMBC-ms8-mf-500-lvl6-species.qzv

In [ ]:
qiime composition da-barplot \
--i-data /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-table-noSC-ANCOMBC-ms8-mf-500-lvl6-reserve.qza \
--p-level-delimiter ';' \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-ANCOMBC-ms8-mf-500-lvl6-reserve.qzv